In [1]:
import numpy as np
import pandas as pd
import re
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook_connected" #jpg, notebook_connected
print(np.__version__)
print(pd.__version__)

1.18.1
1.0.3


***
## Info about the datafile

- The column names and 1 row of sample data is shown below. 
- DB: AS = Aerospike, SQL = MySQL. 
- Schema, Table, FKrepFactor, Constraints: Refer to the paper or presentation slides for the structure of Schemas 1-3

`DB, Schema, Operation, Table, NumRows, FKrepFactor, Constraints, mean, median, stdev, timestamp
AS, 1, Ins, Table1A, 100, 1, True, 1.0257577896118164, 0.9601116180419922, 0.23162600665160718, 2020-03-31_224308`

In [2]:
benchDF = pd.read_csv('../data/NoSQLvsSQL_bench_data.csv', 
             names = ["DB", "Schema", "Operation", "Table", "NumRows", "FKrepFactor", "Constraints", 
                        "mean", "median", "stdev", "timestamp"],
             dtype = {"DB":str, "Schema":int, "Operation":str, "Table":str, "NumRows":int, "FKrepFactor":int, 
                      "Constraints":str, "mean":float, "median":float, "stdev":float, "timestamp":str})

The next two lines of code, `.head()` and `.info()` simply help us get a feel for our data file.

In [3]:
benchDF.head()

,DB,Schema,Operation,Table,NumRows,FKrepFactor,Constraints,mean,median,stdev,timestamp
0,AS,1,Ins,Table1A,100,1,True,1.076329,0.960708,0.454601,2020-03-31_225032
1,AS,1,Ins,Table1B,100,1,True,2.159445,1.910329,0.752107,2020-03-31_225032
2,AS,1,Ins,Table1A,100,1,False,1.058707,0.966311,0.441816,2020-03-31_225033
3,AS,1,Ins,Table1B,100,1,False,1.057017,0.962973,0.493509,2020-03-31_225033
4,AS,1,Upd,Table1A,100,1,False,1.049988,0.950694,0.380215,2020-03-31_225033


In [4]:
benchDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DB           576 non-null    object 
 1   Schema       576 non-null    int32  
 2   Operation    576 non-null    object 
 3   Table        576 non-null    object 
 4   NumRows      576 non-null    int32  
 5   FKrepFactor  576 non-null    int32  
 6   Constraints  576 non-null    object 
 7   mean         576 non-null    float64
 8   median       576 non-null    float64
 9   stdev        576 non-null    float64
 10  timestamp    576 non-null    object 
dtypes: float64(3), int32(3), object(5)
memory usage: 42.9+ KB


***
To simplify other filter and query statements, we'll create 3 dataframes that contain pre-filtered data. One for MySQL data and one each for Aerospike data (with and without constraints enabled).

In [5]:
ASDFConstraintsYes = benchDF[(benchDF["DB"] == "AS") & (benchDF["Constraints"].str.contains("True"))]
ASDFConstraintsNo = benchDF[(benchDF["DB"] == "AS") & (benchDF["Constraints"].str.contains("False"))]
SQLDF = benchDF[benchDF["DB"] == "SQL"]

Verify the size of each resulting dataframe matches what we expect,

In [6]:
print("AeroSpike records WITH constraints:",ASDFConstraintsYes.shape)
print("AeroSpike records WITHOUT constraints:",ASDFConstraintsNo.shape)
print("MySQL records with constraints (implied):",SQLDF.shape)

AeroSpike records WITH constraints: (192, 11)
AeroSpike records WITHOUT constraints: (192, 11)
MySQL records with constraints (implied): (192, 11)


Following a similar operation, we'll define a summary table with measurements only for Aerospike, with constraints, and only at 100K records.

In [7]:
summaryDF = ASDFConstraintsYes.query('NumRows == 100000').drop(columns=["DB", "NumRows", "Constraints", "timestamp"])
# Make Operation a Categorical column so we can sort it.
summaryDF['Operation'] = pd.Categorical(summaryDF['Operation'], categories=[" Ins", " Upd", " Del"])
summaryDF.sort_values(by=["FKrepFactor", "Schema", "Operation", "Table"])

,Schema,Operation,Table,FKrepFactor,mean,median,stdev
432,1,Ins,Table1A,1,1.142562,1.022100,0.954265
433,1,Ins,Table1B,1,2.548501,2.171755,2.082698
438,1,Upd,Table1A,1,1.149836,1.034260,1.161234
439,1,Upd,Table1B,1,1.244729,1.079559,1.632682
440,1,Del,Table1A,1,1.179982,1.032114,0.607112
441,1,Del,Table1B,1,3.587154,3.241539,1.753581
468,2,Ins,Table2A,1,1.149087,1.027822,1.739463
469,2,Ins,Table2B,1,2.375150,2.115726,2.160930
474,2,Upd,Table2A,1,1.130982,1.025200,1.642848
475,2,Upd,Table2B,1,1.158365,1.051426,0.549211


In [8]:
insDF = summaryDF.query('FKrepFactor == 5').drop(columns=["FKrepFactor"]).query('Operation == " Ins"').drop([480])
delDF = summaryDF.query('FKrepFactor == 5').drop(columns=["FKrepFactor"]).query('Operation == " Del"').drop([488, 518, 519])
from IPython.display import display, HTML

# Assuming that dataframes df1 and df2 are already defined:
display(insDF)
display(HTML(delDF.to_html()))

,Schema,Operation,Table,mean,median,stdev
444,1,Ins,Table1A,1.101119,0.993252,1.428704
445,1,Ins,Table1B,3.019018,2.730370,1.737937
481,2,Ins,Table2B,2.883963,2.726078,2.191152
512,3,Ins,TableC,4.668121,4.503489,2.142762


,Schema,Operation,Table,mean,median,stdev
452,1,Del,Table1A,1.052041,0.978708,0.377611
453,1,Del,Table1B,3.322903,3.143549,1.510393
489,2,Del,Table2B,3.239596,2.788305,2.413025
517,3,Del,TableC,4.863379,4.496098,2.753676


***
### Statistics for each of our 3 new dataframes. All times are in mS.

In [9]:
ASDFConstraintsYes[["mean", "median", "stdev"]].describe()

,mean,median,stdev
count,192.000000,192.000000,192.000000
mean,1.980379,1.824597,1.095172
std,1.264896,1.184507,1.121474
min,0.934662,0.913382,0.055409
25%,1.068206,0.994503,0.399630
50%,1.163059,1.036167,0.661322
75%,2.818940,2.668172,1.500443
max,6.437092,5.485892,7.298537


In [10]:
ASDFConstraintsNo[["mean", "median", "stdev"]].describe()

,mean,median,stdev
count,192.000000,192.000000,192.000000
mean,1.097234,0.995890,0.778472
std,0.090822,0.037505,0.814084
min,0.904312,0.895262,0.052312
25%,1.036867,0.970960,0.297875
50%,1.109152,0.998199,0.509287
75%,1.155150,1.020879,0.997779
max,1.456928,1.092672,6.555353


In [11]:
SQLDF[["mean", "median", "stdev"]].describe()

,mean,median,stdev
count,192.000000,192.000000,192.000000
mean,13.461341,13.405674,2.008613
std,19.041243,19.067328,2.650903
min,3.602700,3.602386,0.130901
25%,3.752977,3.711313,0.554544
50%,4.173023,4.123032,1.041048
75%,8.678021,8.576334,1.714058
max,66.482938,66.502094,9.709312


***
Here, we'll define some functions to simplify data extraction and plotting.

In [12]:
def getValues(DF, schema, op, table, FKrep, value):
    schemarepDF = DF.query('Schema==@schema and FKrepFactor==@FKrep')
    opMask = schemarepDF["Operation"].str.contains(op)
    tableMask = schemarepDF["Table"].str.contains(table)
    return schemarepDF[opMask & tableMask].sort_values(by=["NumRows"])[["NumRows", "mean"]]

In [13]:
testDF = getValues(ASDFConstraintsYes, 1, "Ins", "Table1B", 1, "mean")
testDF.head()

,NumRows,mean
1,100,2.159445
145,1000,2.124964
289,10000,2.250650
433,100000,2.548501


In [14]:
def titleToFileName(chgString):
    #remove spaces and punctuation
    removeThese = re.compile(r'[\(\)\s]')
    newString = removeThese.sub("",chgString)
    return newString

In [15]:
AS_Marker = go.scatter.Marker(symbol="circle", size=8)
SQL_Marker = go.scatter.Marker(symbol="square", size=8)
dashLine = go.scatter.Line(dash='dash')

In [16]:
def plot(schema, op, table, FKrep, titleBase, add_legend=True, show_SQL=True, show=True):
    plotS1DF = getValues(ASDFConstraintsYes, schema, op, table, FKrep, "mean")
    plotS2DF = getValues(ASDFConstraintsNo, schema, op, table, FKrep, "mean")
    if show_SQL: plotS3DF = getValues(SQLDF, schema, op, table, FKrep, "mean")
    fig = go.Figure()
    
    if op == "Ins": useInTitle = "Insert"
    if op == "Upd": useInTitle = "Update"
    if op == "Del": useInTitle = "Delete"
    fig.layout.title="Schema "+str(schema)+", "+table+", FK Repetition = "+str(FKrep)+", "+useInTitle+" "+titleBase
    fig.layout.xaxis.title="Number of records in Table"
    fig.layout.xaxis.type="log"
    fig.layout.yaxis.range=[0,70]
    fig.layout.yaxis.title="Time (mS)"

    fig.add_trace(go.Scatter(x=plotS1DF["NumRows"], y=plotS1DF["mean"], 
                             showlegend=add_legend, line_color='teal', marker=AS_Marker, name="Aerospike, Constraints"))
    fig.add_trace(go.Scatter(x=plotS1DF["NumRows"], y=plotS2DF["mean"], 
                             showlegend=add_legend, line_color='navy', marker=AS_Marker, line=dashLine, name="Aerospike, No Constraints"))
    if show_SQL:
        fig.add_trace(go.Scatter(x=plotS1DF["NumRows"], y=plotS3DF["mean"], 
                             showlegend=add_legend, line_color='red', marker=SQL_Marker, name="MySQL"))
        
    if show: 
        fig.show()
        newFileStr = titleToFileName(titleBase)
        if show_SQL:
            imgFile = str("./plots/KVvsSQL_{}_{}_{}.jpg".format(newFileStr, table, op))
        else:
            imgFile = str("./plots/KVwwoCnst_{}_{}_{}.jpg".format(newFileStr, table, op))
        pio.write_image(fig, file=imgFile, format='jpg')
    return fig

In [17]:
def plot_1opAS(schema, op, table, FKrep, titleBase, add_legend=True, show=True):
    plotS1DF = getValues(ASDFConstraintsYes, schema, op, table, FKrep, "mean")
    fig = go.Figure()
    
    if op == "Ins": useInTitle = "Insert"
    if op == "Upd": useInTitle = "Update"
    if op == "Del": useInTitle = "Delete"
    fig.layout.title="Schema "+str(schema)+", "+table+", FK Repetition = "+str(FKrep)+", "+useInTitle+" "+titleBase
    fig.layout.xaxis.title="Number of records in Table"
    fig.layout.xaxis.type="log"
    fig.layout.yaxis.range=[0,70]
    fig.layout.yaxis.title="Time (mS)"

    fig.add_trace(go.Scatter(x=plotS1DF["NumRows"], y=plotS1DF["mean"], 
                             showlegend=add_legend, line_color='teal', marker=AS_Marker, name="Aerospike, Constraints"))
    if show: 
        fig.show()
        newFileStr = titleToFileName(titleBase)
        imgFile = str("./plots/KVOnly_{}_{}.jpg".format(newFileStr, op))
        pio.write_image(fig, file=imgFile, format='jpg')
    return fig

In [18]:
def plotAvsB(sch1, sch2, op, tableA, tableB, FKrep, add_legend = False, show = False):
    plot1DFA = getValues(ASDFConstraintsYes, sch1, op, tableA, FKrep, "mean")
    plot2DFA = getValues(ASDFConstraintsNo, sch1, op, tableA, FKrep, "mean")
    plot1DFB = getValues(ASDFConstraintsYes, sch2, op, tableB, FKrep, "mean")
    plot2DFB = getValues(ASDFConstraintsNo, sch2, op, tableB, FKrep, "mean")
    fig = go.Figure()
    
    if op == "Ins": useInTitle = "Insert"
    if op == "Upd": useInTitle = "Update"
    if op == "Del": useInTitle = "Delete"

    deltas1 = plot1DFB["mean"].reset_index() - plot1DFA["mean"].reset_index() # Note, reset_index() returns DF, not Series
    deltas2 = plot2DFB["mean"].reset_index() - plot2DFA["mean"].reset_index()

    trace1 = dict(type = 'scatter', x=plot1DFA["NumRows"], y=deltas1["mean"],
                  marker=AS_Marker, name="Aerospike, Constraints",
                  line_color = 'teal', showlegend=add_legend)
    trace2 = dict(type = 'scatter', x=plot1DFA["NumRows"], y=deltas2["mean"],
                  marker=AS_Marker, line=dashLine, name="Aerospike, No Constraints", 
                  line_color = 'navy', showlegend=add_legend)
    fig = [trace1, trace2]
    
    if show:
        fig.show()

    return fig

In [19]:
def plotFKreps(schema, op, table, titleBase, AS=True, MySQL=True, customAxis=None, jpgSize=None):
    
    plotS1DF = getValues(ASDFConstraintsYes, schema, op, table, 1, "mean")
    plotS2DF = getValues(ASDFConstraintsYes, schema, op, table, 5, "mean")
    plotS3DF = getValues(ASDFConstraintsYes, schema, op, table, 10, "mean")
    plotS4DF = getValues(SQLDF, schema, op, table, 1, "mean")
    plotS5DF = getValues(SQLDF, schema, op, table, 5, "mean")
    plotS6DF = getValues(SQLDF, schema, op, table, 10, "mean")
    fig = go.Figure()
    
    if op == "Ins": useInTitle = "Insert"
    if op == "Upd": useInTitle = "Update"
    if op == "Del": useInTitle = "Delete"
    fig.layout.title="Schema "+str(schema)+", "+table+", "+useInTitle+" "+titleBase
    fig.layout.xaxis.title="Number of records in Table"
    fig.layout.xaxis.type="log"
    if customAxis != None:
        fig.layout.yaxis.range=customAxis
    else:
        if MySQL:
            fig.layout.yaxis.range=[0,70]
        else:
            fig.layout.yaxis.range=[0,6.5]
    fig.layout.yaxis.title="Time (mS)"

    if AS:
        fig.add_trace(go.Scatter(x=plotS1DF["NumRows"], y=plotS1DF["mean"], name="Aerospike, FKrep = 1", line_color='blue', marker=AS_Marker, mode='lines+markers'))
        fig.add_trace(go.Scatter(x=plotS2DF["NumRows"], y=plotS2DF["mean"], name="Aerospike, FKrep = 5", line_color='teal', marker=AS_Marker, mode='lines+markers'))
        fig.add_trace(go.Scatter(x=plotS3DF["NumRows"], y=plotS3DF["mean"], name="Aerospike, FKrep = 10", line_color='aqua', marker=AS_Marker, mode='lines+markers'))
    if MySQL:
        fig.add_trace(go.Scatter(x=plotS4DF["NumRows"], y=plotS4DF["mean"], name="MySQL, FKrep = 1", line_color='maroon', marker=SQL_Marker, mode='lines+markers'))
        fig.add_trace(go.Scatter(x=plotS5DF["NumRows"], y=plotS5DF["mean"], name="MySQL, FKrep = 5", line_color='red', marker=SQL_Marker, mode='lines+markers'))
        fig.add_trace(go.Scatter(x=plotS6DF["NumRows"], y=plotS6DF["mean"], name="MySQL, FKrep = 10", line_color='fuchsia', marker=SQL_Marker, mode='lines+markers'))
    fig.show()
    newFileStr = titleToFileName(titleBase)
    imgFile = str("./plots/KVvsSQL_{}_{}_{}.jpg".format(newFileStr, table, op))
    if jpgSize == None:
        pio.write_image(fig, file=imgFile, format='jpg')
    else:
        pio.write_image(fig, file=imgFile, format='jpg', width=jpgSize[0], height=jpgSize[1])
    return

***
## Data Plots - Delta Times of Selected Operations and Tables

In [20]:
from plotly import subplots

fig1 = plotAvsB(1, 1, "Ins", "Table1A", "Table1B", 5, add_legend = True)
fig2 = plotAvsB(1, 1, "Upd", "Table1A", "Table1B", 5)
fig3 = plotAvsB(1, 1, "Del", "Table1A", "Table1B", 5)

fig = subplots.make_subplots(rows=3, subplot_titles=["Insert", "Update", "Delete"], shared_xaxes=True, 
                             row_heights=[200,200,200], vertical_spacing=.1)

fig.layout.title="Schema 1, Table1A vs Table1B, PK Verification vs PK and FK Verification"
    
fig.add_traces(fig1, rows=[1, 1], cols=[1, 1])
fig.add_traces(fig2, rows=[2, 2], cols=[1, 1])
fig.add_traces(fig3, rows=[3, 3], cols=[1, 1])
fig.update_yaxes(range=[-0.5,3], title="", row=1, col=1)
fig.update_yaxes(range=[-0.5,3], title="Time Delta (mS)", row=2, col=1)
fig.update_yaxes(range=[-0.5,3], title="", row=3, col=1)
fig.update_xaxes(type="log", title="", row=1, col=1)
fig.update_xaxes(type="log", title="", row=2, col=1)
fig.update_xaxes(type="log", title="Number of records in Table", row=3, col=1)
fig.update_layout()
fig.show()
pio.write_image(fig, file="./plots/KV_Table1A_vs_Table1B_all_ops.jpg", format='jpg')

In [21]:
fig1 = plotAvsB(1, 2, "Ins", "Table1B", "Table2B", 5, add_legend = True)
fig2 = plotAvsB(1, 2, "Upd", "Table1B", "Table2B", 5)
fig3 = plotAvsB(1, 2, "Del", "Table1B", "Table2B", 5)

fig = subplots.make_subplots(rows=3, subplot_titles=["Insert", "Update", "Delete"], shared_xaxes=True, 
                             row_heights=[200,200,200], vertical_spacing=.1)

fig.layout.title="Schema 1 and 2, Table1B vs Table2B, PK Verification vs PK and FK Verification"
    
fig.add_traces(fig1, rows=[1, 1], cols=[1, 1])
fig.add_traces(fig2, rows=[2, 2], cols=[1, 1])
fig.add_traces(fig3, rows=[3, 3], cols=[1, 1])
fig.update_yaxes(range=[-1.5,0.5], title="", row=1, col=1)
fig.update_yaxes(range=[-1.5,0.5], title="Time Delta (mS)", row=2, col=1)
fig.update_yaxes(range=[-1.5,0.5], title="", row=3, col=1)
fig.update_xaxes(type="log", title="", row=1, col=1)
fig.update_xaxes(type="log", title="", row=2, col=1)
fig.update_xaxes(type="log", title="Number of records in Table", row=3, col=1)
fig.update_layout()
fig.show()
pio.write_image(fig, file="./plots/KV_Table1B_vs_Table2B_all_ops.jpg", format='jpg')

In [22]:
fig1 = plotAvsB(2, 3, "Ins", "Table2B", "TableC", 5, add_legend = True)
# No update tests performed for Schema 3
fig2 = plotAvsB(2, 3, "Del", "Table2B", "TableC", 5)

fig = subplots.make_subplots(rows=2, subplot_titles=["Insert", "Delete"], shared_xaxes=True, 
                             row_heights=[200,200], vertical_spacing=.1)

fig.layout.title="Schema 2 and 3, Table2B vs Table(3)C,<br>PK Verification vs Composite PK Verification (includes 2 FKs)"
    
fig.add_traces(fig1, rows=[1, 1], cols=[1, 1])
fig.add_traces(fig2, rows=[2, 2], cols=[1, 1])
fig.update_yaxes(range=[0,4], title="", row=1, col=1)
fig.update_yaxes(range=[0,4], title="Time Delta (mS)", row=2, col=1)
fig.update_xaxes(type="log", title="", row=1, col=1)
fig.update_xaxes(type="log", title="Number of records in Table", row=2, col=1)
fig.update_layout()
fig.show()
pio.write_image(fig, file="./plots/KV_Table2B_vs_TableC_InsDel.jpg", format='jpg')

***
## Data Plots - Measured Times of Operations for each Schema

In [23]:
plot(1, "Ins", "Table1A", 5, "Operation with PK only")
plot(1, "Upd", "Table1A", 5, "Operation with PK only")
plot(1, "Del", "Table1A", 5, "Operation with PK only")
plot(1, "Ins", "Table1B", 5, "Operation with PK and FK (not composite PK)")
plot(1, "Upd", "Table1B", 5, "Operation with PK and FK (not composite PK)")
plot(1, "Del", "Table1B", 5, "Operation with PK and FK (not composite PK)")

In [24]:
fig1 = plot(1, "Ins", "Table1A", 5, "Operation with PK only", show_SQL=False, show=False)
fig2 = plot(1, "Upd", "Table1A", 5, "Operation with PK only", show_SQL=False, show=False)
fig3 = plot(1, "Del", "Table1A", 5, "Operation with PK only", show_SQL=False, show=False)

fig = subplots.make_subplots(rows=3, subplot_titles=["Insert", "Update", "Delete"], shared_xaxes=True, 
                             row_heights=[200,200,200], vertical_spacing=.1)

fig.layout.title="Schema 1, Table1A, PK Verification Only"

trace_count=range(len(fig2.data))
for trace in trace_count:
    fig2.data[trace].showlegend=False
    fig3.data[trace].showlegend=False
    
if len(fig2.data) == 3:
    fig.add_traces(fig1.data, rows=[1, 1, 1], cols=[1, 1, 1])
    fig.add_traces(fig2.data, rows=[2, 2, 2], cols=[1, 1, 1])
    fig.add_traces(fig3.data, rows=[3, 3, 3], cols=[1, 1, 1])
else:
    fig.add_traces(fig1.data, rows=[1, 1], cols=[1, 1])
    fig.add_traces(fig2.data, rows=[2, 2], cols=[1, 1])
    fig.add_traces(fig3.data, rows=[3, 3], cols=[1, 1])
    
fig.update_yaxes(range=[0,1.5], title="", row=1, col=1)
fig.update_yaxes(range=[0,1.5], title="Time (mS)", row=2, col=1)
fig.update_yaxes(range=[0,1.5], title="", row=3, col=1)

fig.update_xaxes(type="log", title="", row=1, col=1)
fig.update_xaxes(type="log", title="", row=2, col=1)
fig.update_xaxes(type="log", title="Number of records in Table", row=3, col=1)

fig.show()
pio.write_image(fig, file="./plots/Table1A_PKverifyOnly_all_ops.jpg", format='jpg')

In [25]:
fig1 = plot(1, "Ins", "Table1B", 5, "Operation with PK and FK (not composite PK)", show_SQL=False, show=False)
fig2 = plot(1, "Upd", "Table1B", 5, "Operation with PK and FK (not composite PK)", show_SQL=False, show=False)
fig3 = plot(1, "Del", "Table1B", 5, "Operation with PK and FK (not composite PK)", show_SQL=False, show=False)

fig = subplots.make_subplots(rows=3, subplot_titles=["Insert", "Update", "Delete"], shared_xaxes=True, 
                             row_heights=[200,200,200], vertical_spacing=.1)

fig.layout.title="Schema 1, Table1B, PK and FK (not composite) Verification"

trace_count=range(len(fig2.data))
for trace in trace_count:
    fig2.data[trace].showlegend=False
    fig3.data[trace].showlegend=False

if len(fig2.data) == 3:
    fig.add_traces(fig1.data, rows=[1, 1, 1], cols=[1, 1, 1])
    fig.add_traces(fig2.data, rows=[2, 2, 2], cols=[1, 1, 1])
    fig.add_traces(fig3.data, rows=[3, 3, 3], cols=[1, 1, 1])
else:
    fig.add_traces(fig1.data, rows=[1, 1], cols=[1, 1])
    fig.add_traces(fig2.data, rows=[2, 2], cols=[1, 1])
    fig.add_traces(fig3.data, rows=[3, 3], cols=[1, 1])

fig.update_yaxes(range=[0,4], title="", row=1, col=1)
fig.update_yaxes(range=[0,4], title="Time (mS)", row=2, col=1)
fig.update_yaxes(range=[0,4], title="", row=3, col=1)

fig.update_xaxes(type="log", title="", row=1, col=1)
fig.update_xaxes(type="log", title="", row=2, col=1)
fig.update_xaxes(type="log", title="Number of records in Table", row=3, col=1)

fig.show()
pio.write_image(fig, file="./plots/Table1B_PKFKverify_all_ops.jpg", format='jpg')

In [26]:
plot(2, "Ins", "Table2B", 5, "Operation with PK and FK (composite PK)")
plot(2, "Upd", "Table2B", 5, "Operation with PK and FK (composite PK)")
plot(2, "Del", "Table2B", 5, "Operation with PK and FK (composite PK)")

In [27]:
fig1 = plot(2, "Ins", "Table2B", 5, "Operation with PK and FK (composite PK)", show=False)
fig2 = plot(2, "Upd", "Table2B", 5, "Operation with PK and FK (composite PK)", show=False)
fig3 = plot(2, "Del", "Table2B", 5, "Operation with PK and FK (composite PK)", show=False)

fig = subplots.make_subplots(rows=3, subplot_titles=["Insert", "Update", "Delete"], shared_xaxes=True, 
                             row_heights=[200,200,200], vertical_spacing=.1)

fig.layout.title="Schema 2, Table2B, PK and FK (composite PK) Verification"

trace_count=range(len(fig2.data))
for trace in trace_count:
    fig2.data[trace].showlegend=False
    fig3.data[trace].showlegend=False

fig.add_traces(fig1.data, rows=[1, 1, 1], cols=[1, 1, 1])
fig.add_traces(fig2.data, rows=[2, 2, 2], cols=[1, 1, 1])
fig.add_traces(fig3.data, rows=[3, 3, 3], cols=[1, 1, 1])

fig.update_yaxes(range=[0,67], title="", row=1, col=1)
fig.update_yaxes(range=[0,67], title="Time (mS)", row=2, col=1)
fig.update_yaxes(range=[0,67], title="", row=3, col=1)

fig.update_xaxes(type="log", title="", row=1, col=1)
fig.update_xaxes(type="log", title="", row=2, col=1)
fig.update_xaxes(type="log", title="Number of records in Table", row=3, col=1)

fig.show()
pio.write_image(fig, file="./plots/Table2B_PKFK_comp_verify_all_ops.jpg", format='jpg')

In [28]:
plot(3, "Ins", "TableC", 5, "Operation with PK and 2 FKs (composite PK)")
plot(3, "Del", "TableC", 5, "Operation with PK and 2 FKs (composite PK)")

In [29]:
fig1 = plot(3, "Ins", "TableC", 5, "Operation with PK and 2 FKs (composite PK)", show_SQL=False, show=False)
fig2 = plot(3, "Del", "TableC", 5, "Operation with PK and 2 FKs (composite PK)", show_SQL=False, show=False)

fig = subplots.make_subplots(rows=2, subplot_titles=["Insert", "Delete"], shared_xaxes=True, 
                             row_heights=[200,200], vertical_spacing=.1)

fig.layout.title="Schema 3, TableC, PK and 2 FKs (composite PK) Verification"

fig.update_yaxes(range=[0,6.5], title="", row=1, col=1)
fig.update_yaxes(range=[0,6.5], title="Time (mS)", row=2, col=1)

fig.update_xaxes(type="log", title="", row=1, col=1)
fig.update_xaxes(type="log", title="Number of records in Table", row=2, col=1)

trace_count=range(len(fig2.data))
for trace in trace_count:
    fig2.data[trace].showlegend=False

if len(fig2.data) == 3:
    fig.add_traces(fig1.data, rows=[1, 1, 1], cols=[1, 1, 1])
    fig.add_traces(fig2.data, rows=[2, 2, 2], cols=[1, 1, 1])
    pio.write_image(fig, file="./plots/Table3C_PK2FK_comp_verify_all_ops.jpg", format='jpg')
else:
    fig.add_traces(fig1.data, rows=[1, 1], cols=[1, 1])
    fig.add_traces(fig2.data, rows=[2, 2], cols=[1, 1])


fig.show()

In [30]:
plotFKreps(2, "Del", "Table2B", "Effect of FK repetition", jpgSize=[700,300])
plotFKreps(3, "Del", "TableC", "Effect of FK repetition", jpgSize=[700,300])

plotFKreps(2, "Ins", "Table2B", "Effect of FK repetition", customAxis=[0,9])
plotFKreps(2, "Del", "Table2B", "Effect of FK repetition (zoom)", customAxis=[0,9])
plotFKreps(3, "Ins", "TableC", "Effect of FK repetition", customAxis=[0,9])
plotFKreps(3, "Del", "TableC", "Effect of FK repetition (zoom)", customAxis=[0,9])

In [31]:
plotFKreps(2, "Ins", "Table2B", "Effect of FK repetition, Aerospike only", AS=True, MySQL=False)
plotFKreps(2, "Del", "Table2B", "Effect of FK repetition, Aerospike only", AS=True, MySQL=False)
plotFKreps(3, "Ins", "TableC", "Effect of FK repetition, Aerospike only", AS=True, MySQL=False)
plotFKreps(3, "Del", "TableC", "Effect of FK repetition, Aerospike only", AS=True, MySQL=False)

In [32]:
def plot_op_allSchemas(op, FKrep=5):
    add_legend = True

    plotS1DF = getValues(ASDFConstraintsYes, 1, op, "Table1A", FKrep, "mean")
    plotS1BDF =getValues(ASDFConstraintsYes, 1, op, "Table1B", FKrep, "mean")
    plotS2DF = getValues(ASDFConstraintsYes, 2, op, "Table2B", FKrep, "mean")
    plotS3DF = getValues(ASDFConstraintsYes, 3, op, "TableC", FKrep, "mean")
    fig = go.Figure()

    if op == "Ins": 
        useInTitle = "Insert"
        fig.layout.title="Aerospike<br>Schemas 1-3, "+useInTitle+", Tables 1A, 1B, 2B, 3C, FK Repetition = "+str(FKrep)
    if op == "Upd": 
        useInTitle = "Update"
        fig.layout.title="Aerospike<br>Schemas 1-2, "+useInTitle+", Tables 1A, 1B, 2B, FK Repetition = "+str(FKrep)
    if op == "Del": 
        useInTitle = "Delete"
        fig.layout.title="Aerospike<br>Schemas 1-3, "+useInTitle+", Tables 1A, 1B, 2B, 3C, FK Repetition = "+str(FKrep)
    
    fig.layout.xaxis.title="Number of records in Table"
    fig.layout.xaxis.type="log"
    fig.layout.yaxis.range=[1,6.5]
    fig.layout.yaxis.title="Time (mS)"

    fig.add_trace(go.Scatter(x=plotS1DF["NumRows"], y=plotS1DF["mean"], 
                             showlegend=add_legend, line_color='teal', marker=AS_Marker, name="Table1A"))
    fig.add_trace(go.Scatter(x=plotS1BDF["NumRows"], y=plotS1BDF["mean"], 
                             showlegend=add_legend, line_color='teal', marker=AS_Marker, name="Table1B", line=dashLine))
    fig.add_trace(go.Scatter(x=plotS2DF["NumRows"], y=plotS2DF["mean"], 
                             showlegend=add_legend, line_color='navy', marker=AS_Marker, name="Table2B"))
    fig.add_trace(go.Scatter(x=plotS3DF["NumRows"], y=plotS3DF["mean"], 
                             showlegend=add_legend, line_color='maroon', marker=AS_Marker, name="Table3C"))

    fig.show()
    imgFile = str("./plots/KV_Cnst_TblAll_AS_{}.jpg".format(op))
    pio.write_image(fig, file=imgFile, format='jpg')
    
plot_op_allSchemas("Ins")
plot_op_allSchemas("Upd")
plot_op_allSchemas("Del")